<a href="https://colab.research.google.com/github/ShahidHasib586/Underwater-object-detection-using-waternet-and-yolov7/blob/main/Code/yolov7/workingai_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Check if CUDA is available for tensorflow backend (tensorflow is not required its just for cheking my environment)

In [ ]:
import tensorflow as tf

# Check if TensorFlow can access GPU
if tf.test.is_built_with_cuda():
    print("TensorFlow is built with CUDA support.")
else:
    print("TensorFlow is not built with CUDA support.")

# Check if GPU devices are available
if tf.config.list_physical_devices('GPU'):
    print("GPU is available.")
    print("TensorFlow GPU devices:", tf.config.list_physical_devices('GPU'))
else:
    print("No GPU available in TensorFlow.")


TensorFlow is built with CUDA support.
GPU is available.
TensorFlow GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


Check if torch is working with CUDA

In [ ]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available.")
    print("PyTorch is using GPU:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available.")


CUDA is available.
PyTorch is using GPU: NVIDIA GeForce RTX 4060 Laptop GPU


Check tensorflow is properly utilizing the GPU

In [ ]:
import tensorflow as tf
import time

# Ensure GPU is being used
print("Is TensorFlow using GPU? ", tf.config.experimental.list_physical_devices('GPU'))

# Define a large matrix
matrix_size = 1000
a = tf.random.normal((matrix_size, matrix_size))
b = tf.random.normal((matrix_size, matrix_size))

# Perform matrix multiplication on GPU and measure time
start_time = time.time()
result = tf.matmul(a, b)
end_time = time.time()

print("Matrix multiplication result shape:", result.shape)
print(f"Time taken on GPU: {end_time - start_time:.4f} seconds")


Is TensorFlow using GPU?  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Matrix multiplication result shape: (1000, 1000)
Time taken on GPU: 0.8816 seconds


Check pytorch is properly utilizing the GPU

In [ ]:
import torch
import time

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Define a large matrix
matrix_size = 1000
a = torch.randn(matrix_size, matrix_size, device=device)
b = torch.randn(matrix_size, matrix_size, device=device)

# Perform matrix multiplication on GPU and measure time
start_time = time.time()
result = torch.mm(a, b)
end_time = time.time()

print("Matrix multiplication result shape:", result.shape)
print(f"Time taken on GPU: {end_time - start_time:.4f} seconds")


Using device: cuda
Matrix multiplication result shape: torch.Size([1000, 1000])
Time taken on GPU: 0.0238 seconds


## Git clone the YOLOv7 repo and install its dependencies.

In [ ]:
# Clone YOLOv7 repository
!git clone https://github.com/WongKinYiu/yolov7.git

# Install dependencies
!pip install -r requirements.txt


fatal: destination path 'yolov7' already exists and is not an empty directory.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


Change the directiory to yolov7

In [ ]:
%cd yolov7

C:\Users\shahi\yolov7


Check if the images are formatted properly

In [ ]:
import os

# Paths to train, valid, and test image folders
image_dirs = [
    r"C:\Users\shahi\Downloads\underwater project data\aquarium_pretrain\train\images",
    r"C:\Users\shahi\Downloads\underwater project data\aquarium_pretrain\valid\images",
    r"C:\Users\shahi\Downloads\underwater project data\aquarium_pretrain\test\images",
]

# Check image formats
for image_dir in image_dirs:
    print(f"Checking images in {image_dir}...")
    for image_file in os.listdir(image_dir):
        if not (image_file.endswith(".jpg") or image_file.endswith(".png")):
            print(f"Invalid file: {image_file}")
print("Image format check complete.")


Checking images in C:\Users\shahi\Downloads\underwater project data\aquarium_pretrain\train\images...
Checking images in C:\Users\shahi\Downloads\underwater project data\aquarium_pretrain\valid\images...
Checking images in C:\Users\shahi\Downloads\underwater project data\aquarium_pretrain\test\images...
Image format check complete.


Check if images are labbled properly

In [ ]:
import os

# Paths to train, valid, and test label folders
label_dirs = [
    r"C:\Users\shahi\Downloads\underwater project data\aquarium_pretrain\train\labels",
    r"C:\Users\shahi\Downloads\underwater project data\aquarium_pretrain\valid\labels",
    r"C:\Users\shahi\Downloads\underwater project data\aquarium_pretrain\test\labels",
]

# Validate label format
for label_dir in label_dirs:
    print(f"Checking labels in {label_dir}...")
    for label_file in os.listdir(label_dir):
        if not label_file.endswith(".txt"):
            print(f"Invalid label file: {label_file}")
            continue

        file_path = os.path.join(label_dir, label_file)
        with open(file_path, "r") as f:
            for line_num, line in enumerate(f, start=1):
                try:
                    parts = line.strip().split()
                    if len(parts) != 5:
                        raise ValueError("Line does not have exactly 5 elements")
                    class_id = int(parts[0])
                    coords = list(map(float, parts[1:]))
                    if not (0 <= class_id < 7):  # Assuming 7 classes
                        raise ValueError("Invalid class ID")
                    if not all(0 <= coord <= 1 for coord in coords):
                        raise ValueError("Coordinates not normalized")
                except Exception as e:
                    print(f"Error in {label_file}, line {line_num}: {e}")
print("Label format check complete.")


Checking labels in C:\Users\shahi\Downloads\underwater project data\aquarium_pretrain\train\labels...
Checking labels in C:\Users\shahi\Downloads\underwater project data\aquarium_pretrain\valid\labels...
Checking labels in C:\Users\shahi\Downloads\underwater project data\aquarium_pretrain\test\labels...
Label format check complete.


Check if images matches the labels properly

In [ ]:
for image_dir, label_dir in zip(image_dirs, label_dirs):
    print(f"Checking matching files in {image_dir} and {label_dir}...")
    image_files = {os.path.splitext(f)[0] for f in os.listdir(image_dir)}
    label_files = {os.path.splitext(f)[0] for f in os.listdir(label_dir)}

    missing_labels = image_files - label_files
    missing_images = label_files - image_files

    if missing_labels:
        print(f"Missing label files: {missing_labels}")
    if missing_images:
        print(f"Missing image files: {missing_images}")
print("Matching files check complete.")


Checking matching files in C:\Users\shahi\Downloads\underwater project data\aquarium_pretrain\train\images and C:\Users\shahi\Downloads\underwater project data\aquarium_pretrain\train\labels...
Checking matching files in C:\Users\shahi\Downloads\underwater project data\aquarium_pretrain\valid\images and C:\Users\shahi\Downloads\underwater project data\aquarium_pretrain\valid\labels...
Checking matching files in C:\Users\shahi\Downloads\underwater project data\aquarium_pretrain\test\images and C:\Users\shahi\Downloads\underwater project data\aquarium_pretrain\test\labels...
Matching files check complete.


There is a problem with my model run because of version mismatch, remove all the packages and reinstall them to see if CUDA is working properly this time

In [ ]:
import torch
import torchvision
import torchaudio

print(f"PyTorch version: {torch.__version__}")
print(f"torchvision version: {torchvision.__version__}")
print(f"torchaudio version: {torchaudio.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")


PyTorch version: 2.1.2+cu121
torchvision version: 0.16.2+cu121
torchaudio version: 2.1.2+cu121
CUDA available: True


RUN a simple test

In [ ]:
import torch

# Check CUDA device
print("CUDA Device Count:", torch.cuda.device_count())
print("Current CUDA Device:", torch.cuda.current_device())
print("Device Name:", torch.cuda.get_device_name(0))

# Perform a simple tensor operation on the GPU
a = torch.rand(3, 3).cuda()
b = torch.rand(3, 3).cuda()
print("Tensor addition result on GPU:", a + b)


CUDA Device Count: 1
Current CUDA Device: 0
Device Name: NVIDIA GeForce RTX 4060 Laptop GPU
Tensor addition result on GPU: tensor([[0.4564, 0.8435, 1.0889],
        [1.0319, 0.8137, 1.8275],
        [0.3101, 1.1559, 1.1078]], device='cuda:0')


RUN a simple test on  torchaudio

In [ ]:
import torchaudio
from torchaudio.transforms import Resample

# Test with a simple tensor
waveform = torch.rand(1, 16000)  # Dummy waveform
resampler = Resample(orig_freq=16000, new_freq=8000)
resampled_waveform = resampler(waveform)

print("Resampled waveform shape:", resampled_waveform.shape)


Resampled waveform shape: torch.Size([1, 8000])


## Disable WANDB

In [ ]:
#!pip install tensorboard
import os
os.environ["WANDB_MODE"] = "disabled"



Update some versions

In [ ]:
#!pip uninstall torch torchvision torchaudio -y
#!pip install torch==2.1.2+cu121 torchvision==0.16.2+cu121 torchaudio==2.1.2+cu121 -f https://download.pytorch.org/whl/torch_stable.html


Found existing installation: torch 2.1.2+cu121
Uninstalling torch-2.1.2+cu121:
  Successfully uninstalled torch-2.1.2+cu121
Found existing installation: torchvision 0.16.2+cu121
Uninstalling torchvision-0.16.2+cu121:
  Successfully uninstalled torchvision-0.16.2+cu121
Found existing installation: torchaudio 2.1.2+cu121
Uninstalling torchaudio-2.1.2+cu121:
  Successfully uninstalled torchaudio-2.1.2+cu121
Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu121/torch-2.1.2%2Bcu121-cp39-cp39-win_amd64.whl (2473.9 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchvision-0.16.2%2Bcu121-cp39-cp39-win_amd64.whl (5.6 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.1.2%2Bcu121-cp39-cp39-win_amd64.whl (4.0 MB)


RUN a simple train

In [ ]:
!python train.py --img 640 --batch 8 --epochs 10 --data data.yaml --cfg cfg/training/yolov7.yaml --weights yolov7.pt --device 0



autoanchor: Analyzing anchors... anchors/target = 5.03, Best Possible Recall (BPR) = 0.9991
                 all         127         909      0.0037      0.0151     0.00061    0.000126
                 all         127         909      0.0259      0.0628     0.00871     0.00195
                 all         127         909      0.0547       0.244      0.0384     0.00951
                 all         127         909       0.206       0.232       0.135      0.0534
                 all         127         909       0.144       0.341        0.14      0.0508
                 all         127         909        0.35        0.26       0.211      0.0918
                 all         127         909        0.25        0.39       0.242       0.103
                 all         127         909       0.316       0.383       0.281       0.114
                 all         127         909       0.404       0.422       0.342       0.174
                 all         127         909       0.356       0.493  

YOLOR  v0.1-128-ga207844 torch 2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8187.5MB)

Namespace(weights='yolov7.pt', cfg='cfg/training/yolov7.yaml', data='.\\data\\data.yaml', hyp='data/hyp.scratch.p5.yaml', epochs=10, batch_size=8, img_size=[640, 640], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='0', multi_scale=False, single_cls=False, adam=False, sync_bn=False, local_rank=-1, workers=8, project='runs/train', entity=None, name='exp', exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias='latest', freeze=[0], v5_metric=False, world_size=1, global_rank=-1, save_dir='runs\\train\\exp8', total_batch_size=8)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_

In [ ]:
!python train.py --img 640 --batch 4 --epochs 2 --data data.yaml --cfg cfg/training/yolov7.yaml --weights yolov7.pt --device 0



autoanchor: Analyzing anchors... anchors/target = 5.03, Best Possible Recall (BPR) = 0.9991
                 all         127         909     0.00966      0.0254     0.00222    0.000449
                 all         127         909      0.0546      0.0528      0.0177     0.00484
                fish         127         459      0.0723       0.229       0.048     0.00951
           jellyfish         127         155      0.0138     0.00645     0.00437    0.000844
             penguin         127         104      0.0632     0.00962     0.00592     0.00144
              puffin         127          74      0.0859      0.0135      0.0072     0.00129
               shark         127          57           0           0     0.00239     0.00067
            starfish         127          27       0.147       0.111      0.0541      0.0196
            stingray         127          33           0           0     0.00206    0.000543
Optimizer stripped from runs\train\exp9\weights\last.pt, 74.8MB
Optimi

YOLOR  v0.1-128-ga207844 torch 2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8187.5MB)

Namespace(weights='yolov7.pt', cfg='cfg/training/yolov7.yaml', data='.\\data\\data.yaml', hyp='data/hyp.scratch.p5.yaml', epochs=2, batch_size=4, img_size=[640, 640], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='0', multi_scale=False, single_cls=False, adam=False, sync_bn=False, local_rank=-1, workers=8, project='runs/train', entity=None, name='exp', exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias='latest', freeze=[0], v5_metric=False, world_size=1, global_rank=-1, save_dir='runs\\train\\exp9', total_batch_size=4)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_m

In [ ]:
!python detect.py --weights runs/train/exp8/weights/best.pt --img 640 --conf 0.25 --source path/test/images


Namespace(weights=['runs/train/exp8/weights/best.pt'], source='path/test/images', img_size=640, conf_thres=0.25, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

Done. (23.3ms) Inference, (21.0ms) NMS
 The image with the result is saved in: runs\detect\exp8\IMG_2289_jpeg_jpg.rf.fe2a7a149e7b11f2313f5a7b30386e85.jpg
6 penguins, Done. (29.5ms) Inference, (20.3ms) NMS
 The image with the result is saved in: runs\detect\exp8\IMG_2301_jpeg_jpg.rf.2c19ae5efbd1f8611b5578125f001695.jpg
6 penguins, Done. (20.0ms) Inference, (0.0ms) NMS
 The image with the result is saved in: runs\detect\exp8\IMG_2319_jpeg_jpg.rf.6e20bf97d17b74a8948aa4877

YOLOR  v0.1-128-ga207844 torch 2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8187.5MB)

C:\Users\shahi\anaconda3\envs\env_3\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3527.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 314 layers, 36514136 parameters, 6194944 gradients, 103.3 GFLOPS
